In [1]:
import os, re 
import numpy as np
import tensorflow as tf

import glob
import os

In [2]:

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 데이터 정제하기

공백 제거 및 일정 길이 이상 문장 제외하기

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
print(corpus[:10])

print(len(corpus))

['<start> now i ve heard there was a secret chord <end>', '<start> that david played , and it pleased the lord <end>', '<start> but you don t really care for music , do you ? <end>', '<start> it goes like this <end>', '<start> the fourth , the fifth <end>', '<start> the minor fall , the major lift <end>', '<start> the baffled king composing hallelujah hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah your faith was strong but you needed proof <end>']
175986


In [6]:
corpus[-1]

'<start> you know , i love you <end>'

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
 
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 12000단어를 포함되지 못한 단어는 '<unk>'로 바꿈
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus) 
    print(tensor[50])
    
    check_tensor=[]
    for temp in tensor:
        if len(temp)<=15 and len(temp)!=0:
            check_tensor.append(temp)
    
    print(len(check_tensor))
        
    
    check_tensor = tf.keras.preprocessing.sequence.pad_sequences(check_tensor, padding='post')  
    
   #print(tensor,tokenizer)
    return check_tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[2, 8, 36, 46, 7, 243, 10, 92, 69, 17, 7, 76, 38, 33, 10, 114, 69, 3]
156227


In [8]:
print(len(tensor))
print(len(tensor[50]))

156227
15


In [9]:

# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0]) # 2<start>에서 시작해서 3<enc>으로 끝난 후 0<pad>로 채워져있음
print(tgt_input[0]) # 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가짐

[   2   50    5   91  297   64   57    9  970 6048    3    0    0    0]
[  50    5   91  297   64   57    9  970 6048    3    0    0    0    0]


### dataset 생성 후 train, validation, test 분리

In [10]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val =  train_test_split(src_input,tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)

print('enc_train 개수: ', len(enc_train),', enc_val 개수: ', len(dec_train))

enc_train 개수:  124981 , enc_val 개수:  31246


In [11]:
dec_train.shape

(124981, 14)

In [12]:
BUFFER_SIZE = len(dec_train)
BATCH_SIZE = 256
steps_per_epoch = len(dec_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [18]:

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.0429641e-05, -1.4360924e-04,  2.4599309e-05, ...,
         -2.7488417e-04,  2.6435123e-06,  1.5015624e-04],
        [ 7.5417556e-06,  1.5241770e-06,  1.2106828e-04, ...,
         -3.8726590e-04,  3.9834144e-05,  3.1748426e-04],
        [ 1.2329988e-04,  3.3919339e-05,  1.6426327e-04, ...,
         -3.6112964e-04,  3.6226975e-05,  4.2849442e-04],
        ...,
        [-1.2370163e-04,  1.0195761e-04, -1.4808911e-04, ...,
          3.4373658e-04,  2.4755215e-04,  6.3600997e-04],
        [-3.3638594e-04,  5.1374675e-05, -4.4791683e-04, ...,
          5.7998383e-05,  5.2318932e-04,  2.9383009e-04],
        [-4.4595121e-04, -2.6003077e-05, -7.1282295e-04, ...,
         -2.6101634e-04,  7.6508662e-04, -8.8591936e-05]],

       [[ 2.0429641e-05, -1.4360924e-04,  2.4599309e-05, ...,
         -2.7488417e-04,  2.6435123e-06,  1.5015624e-04],
        [-9.9516605e-05, -1.6241649e-05,  1.5214898e-04, ...,
         -2.6733731e-04, 

In [19]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [21]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
610/610 [==============================] - 103s 165ms/step - loss: 3.4192
Epoch 2/10
610/610 [==============================] - 104s 170ms/step - loss: 2.9690
Epoch 3/10
610/610 [==============================] - 104s 170ms/step - loss: 2.7944
Epoch 4/10
610/610 [==============================] - 104s 170ms/step - loss: 2.6638
Epoch 5/10
610/610 [==============================] - 104s 171ms/step - loss: 2.5539
Epoch 6/10
610/610 [==============================] - 104s 171ms/step - loss: 2.4556
Epoch 7/10
610/610 [==============================] - 104s 171ms/step - loss: 2.3653
Epoch 8/10
610/610 [==============================] - 104s 171ms/step - loss: 2.2810
Epoch 9/10
610/610 [==============================] - 104s 171ms/step - loss: 2.2017
Epoch 10/10
610/610 [==============================] - 104s 171ms/step - loss: 2.1268


In [22]:
# 모델이 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '